In [2]:
pip install pandas psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install --upgrade pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 27.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3


In [1]:
import pandas as pd
import psycopg2 as pg

connection = pg.connect(database='clever',user='clever',host='postgres_clever',port='5432',password='clever')

df_fmcsa = pd.read_sql('select usdot_num,company_name,registered_address,phone_number from public.fmcsa_company_snapshot', connection)

df_google = pd.read_sql('select google_id,name, full_address, phone from company_profiles_google_maps',connection)


/tmp/ipykernel_4815/2630579525.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fmcsa = pd.read_sql('select usdot_num,company_name,registered_address,phone_number from public.fmcsa_company_snapshot', connection)
/tmp/ipykernel_4815/2630579525.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_google = pd.read_sql('select google_id,name, full_address, phone from company_profiles_google_maps',connection)


In [8]:
df_fmcsa.head()

,usdot_num,company_name,registered_address,phone_number
0,3975699,GOING GOLDENN TRANSPORTATION LLC,"4420 MARK TRAIL WAY\r\nDALLAS, TX 75232-1047",(214)709-8984
1,4040772,KINECT TRANSPORT LLC,"14155 DALLAS PKWY APT 1543\r\nDALLAS, TX 75254",(501)358-2015
2,4059470,GULED JAMA,"7509 24TH AVE SW APT 3\r\nSEATTLE, WA 98106",(206)335-8042
3,4054636,E&M EXPRESS LOGISTICS LLC,"750 N SAINT PAUL ST STE 250\r\nDALLAS, TX 75201",(469)396-1815
4,4054993,RELIASAFE DELIVERIES,"9511 SW 81ST ST\r\nMIAMI, FL 33173",(305)582-6901


In [9]:
df_google.head()

,google_id,name,full_address,phone
0,0x111f53bf849f031:0xb43eff4f62923c5d,Nicolas Boucher P.A,"901 S Miami Ave #215, Miami, FL 33130",+1 305-788-8035
1,0x14e179155727173b:0x2c342e5f13a9b2d,New Hope Realty,"110 Evans Mill Dr STE 102, Dallas, GA 30157",+1 770-635-7560
2,0x14e4b9f6c46524a9:0x202749ab2be6bfe3,Harvey Mclean & Associates,"4922 Abbott Ave, Dallas, TX 75219",+1 214-219-4555
3,0x1803389f251373:0x26e9e6cc7312627c,Yanet Mino Realtor,"7281 Coral Wy, Miami, FL 33155",+1 305-570-8861
4,0x18751cf7cb7e9a5:0xad85fc0dfe9ff490,Lunabase Travelstays & Property Management,"1600 NE 1st Ave Unit 3419, Miami, FL 33132",+1 305-204-8717


In [2]:
import re

def clean_phone_number(phone):
    if not phone:
        return ""
    # Remove the +1 prefix if present
    phone = re.sub(r'^\+1\s*', '', phone)
    # Remove all non-numeric characters
    phone = re.sub(r'\D', '', phone)
    return phone

In [3]:
df_fmcsa['treated_phone'] = df_fmcsa['phone_number'].apply(clean_phone_number)
df_google['treated_phone'] = df_google['phone'].apply(clean_phone_number)

In [4]:
df_fmcsa=df_fmcsa.fillna("")
df_google=df_google.fillna("")

In [5]:
df_fmcsa['fuzzy_column'] = df_fmcsa['treated_phone'] + ' ' + df_fmcsa['registered_address'] + ' ' + df_fmcsa['company_name']
df_google['fuzzy_column'] = df_google['treated_phone'] + ' ' + df_google['full_address'] + ' ' + df_google['name']

In [6]:
df_google['fuzzy_column'] = df_google['fuzzy_column'].str.upper().replace(r'[\r\t]', ' ', regex=True)
df_fmcsa['fuzzy_column'] = df_fmcsa['fuzzy_column'].str.upper().replace(r'[\r\t]', ' ', regex=True)

In [7]:
df_fmcsa[df_fmcsa['fuzzy_column'].isna()]

,usdot_num,company_name,registered_address,phone_number,treated_phone,fuzzy_column


In [63]:
merged_df = pd.merge(df_fmcsa[~df_fmcsa['phone_number'].isna()], df_google[~df_google['phone'].isna()], on='treated_phone', how='inner')

In [64]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   usdot_num           2191 non-null   int64 
 1   company_name        2191 non-null   object
 2   registered_address  2191 non-null   object
 3   phone_number        2191 non-null   object
 4   treated_phone       2191 non-null   object
 5   fuzzy_column_x      2191 non-null   object
 6   google_id           2191 non-null   object
 7   name                2191 non-null   object
 8   full_address        2191 non-null   object
 9   phone               2191 non-null   object
 10  fuzzy_column_y      2191 non-null   object
dtypes: int64(1), object(10)
memory usage: 188.4+ KB


In [43]:
pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Function to perform fuzzy matching based on combined_info
def fuzzy_merge(df_left, df_right, key_left, key_right, threshold=70, limit=1):
    """
    Fuzzy match and merge two dataframes based on a specific key column from both.
    
    df_left: Left DataFrame
    df_right: Right DataFrame
    key_left: Key column in left DataFrame to match on
    key_right: Key column in right DataFrame to match on
    threshold: Minimum matching score (0-100)
    limit: Maximum number of matches to return
    
    Returns: Merged DataFrame
    """
    s = df_right[key_right].tolist()
    
    matches = []
    for idx, row in df_left.iterrows():
        match = process.extractOne(row[key_left], s, scorer=fuzz.token_sort_ratio)
        
        if match and match[1] >= threshold:
            matched_row = df_right[df_right[key_right] == match[0]].iloc[0]
            match_data = {
                'left_combined_info': row[key_left],
                'right_combined_info': match[0],
                'match_score': match[1]
            }
            match_data.update(row.to_dict())
            match_data.update(matched_row.to_dict())
            matches.append(match_data)
    
    return pd.DataFrame(matches)


In [9]:
merged_df = fuzzy_merge(df_fmcsa, df_google, key_left='fuzzy_column', key_right='fuzzy_column', threshold=73)

In [10]:
merged_df[['usdot_num','google_id']]

,usdot_num,google_id
0,3227005,0x5490157655555555:0x673e5a56019f797e
1,3575039,0x864e99b6a000bbab:0x4f409783fb0da34f
2,2438489,0x88d9ba34781b0771:0xa2b03cc26eee9d6
3,1934023,0x5490156e47c9aeb1:0x695b4b7ae4531d49
4,2401554,0x864e9f0b4ad4f241:0x28134aed8a1286b1
5,2298043,0x864e9e6052a57db3:0x6ae7757913fe350e
6,1748105,0x88d9ae3866fbb4bf:0xdf69ef0c0f050d63
7,1475638,0x88d9ae3866fbb4bf:0xdf69ef0c0f050d63
8,1413859,0x864c2790527c79f9:0x5c15f6893fc59a3e
9,2857394,0x864e9c00a99fa229:0xeb59fd561e55c274


merged_df